### Topic modeling
Extracting topics from the reddit messages 

##### Imports 

In [1]:
# Imports
from bertopic import BERTopic
import pandas as pd
import numpy as np
import nltk 

##### Data 

Loading the data 

In [2]:
# getting subset of data 
messages_df = pd.read_csv("reddit_22_51/messages.csv", sep="\t")
messages_df.head()

,id,user,text
0,j0s252k,HexagonOfVirtue,"im gonna find it just to check, it's not the ..."
1,j0s25h2,Teephex,According to you criticizing and being skeptic...
2,j0s25ht,1platesquat,Gotcha. Can you explain to me why your opinion...
3,j0s25l5,YouLostTheGame,"Euros, which some argue is actually harder tha..."
4,j0s25nr,HMID_Delenda_Est,You've been sounding more like PunishedSubSist...


Info on the data 

In [3]:
# info on data
column_list = messages_df.columns
shape = messages_df.shape

print("columns: ", column_list)
print("shape ", shape)

columns:  Index(['id', 'user', 'text'], dtype='object')
shape  (290898, 3)


**Data cleaning: stop word removal**

Stop word are very frequent words e.g. “the” and “a” that can impact the topics generated by the bertopic model due to their high frequency across most documents/texts - can be removed to get clearer more informative topics 

In [4]:
# METHOD 1: use a predefined list of stop words 
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stop_words(text, stop_words): 
    words = text.split() 
    # Use a list comprehension to remove stop words 
    filtered_words = [word for word in words if word.lower() not in stop_words] 
  
  # Join the filtered words back into a sentence 
    return ' '.join(filtered_words)

messages_df_sw = messages_df.copy()
messages_df_sw["text"] = messages_df_sw["text"].apply(lambda row: remove_stop_words(row, stop_words))
messages_df_sw.head()

[nltk_data] Downloading package stopwords to /home/jnye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,user,text
0,j0s252k,HexagonOfVirtue,"im gonna find check, ai art one right? trigger..."
1,j0s25h2,Teephex,According criticizing skeptical active politic...
2,j0s25ht,1platesquat,Gotcha. explain opinion important me?
3,j0s25l5,YouLostTheGame,"Euros, argue actually harder WC"
4,j0s25nr,HMID_Delenda_Est,sounding like PunishedSubSister tbh


In [5]:
# METHOD 2: remove most frequent words found in the data 
most_frequent_words = pd.Series(' '.join(messages_df['text']).lower().split()).value_counts()[:100].index.tolist()

messages_df_sw2 = messages_df.copy()
messages_df_sw2["text"] = messages_df_sw2["text"].apply(lambda row: remove_stop_words(row, most_frequent_words))

Convert data to list of strings (input needed by bertopic)

In [6]:
# convert to list of strings (input needed by bertopic model)
messages_list = messages_df["text"].astype(str).tolist()
len(messages_list)

290898

Get subset of data to work with 

In [7]:
# subset of data - 5000 rows
messages_subset = messages_list[:10000]
messages_subset

["im gonna find it  just to check, it's not the ai art one right? it's triggered before but rare",
 'According to you criticizing and being skeptical of an active politican is a mental disorder?',
 'Gotcha. Can you explain to me why your opinion should be important to me?',
 'Euros, which some argue is actually harder than a WC',
 "You've been sounding more like PunishedSubSister tbh",
 'If you hate people for their race you hate people for how they are born',
 'That communism is an abysmal failure in ever single instances in which it has been attempted? Yeah. Hans would agree.',
 'Have you made a genuine effort to try and estimate the cost of moving to a more affordable place?',
 "He implemented a rule saying that you couldn't link to other social media sites (Instagram, facebook, Mastodon, etc)",
 'Yes. Every single human on earth eats tiny amounts of poison on a regular basis. You know why I know that? Because literally everything is poisonous depending on the dose. Water itself is 

##### Bertopic model 

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english")

# topic_model = BERTopic()
topic_model = BERTopic(vectorizer_model=CountVectorizer(stop_words="english")) 
# topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

NameError: name 'BERTopic' is not defined

In [9]:
# fitting the bertopic model 
topic_model_fitted = topic_model.fit(messages_subset)

In [10]:
# parameters of the general model
topic_model_fitted.get_params()

{'calculate_probabilities': False,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': <bertopic.backend._sentencetransformers.SentenceTransformerBackend at 0x7f6d0eb83640>,
 'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 1),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, low_memory=False, metric='cosine', min_dist=0.0, n_components=5, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(),
 'verbose': False,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [11]:
# get info on topics (names, important words, representative reddit message/document)
topic_info = topic_model_fitted.get_topic_info()
topic_info.to_csv("topic_info_10000_sklearn.csv", sep="\t", index=False)

In [12]:
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4748,-1_the_to_of_and,"[the, to, of, and, that, you, it, is, in, for]","[ To a certain extent, it is possible, but I t..."
1,0,128,0_sorry_harrumph_heafty_whimpy,"[sorry, harrumph, heafty, whimpy, im, heh, hah...",[I’m sorry to hear that Are you doing alright...
2,1,126,1_abortion_fetus_pregnancy_sex,"[abortion, fetus, pregnancy, sex, unborn, life...","[ Yeah, fair point, badly worded on my part. B..."
3,2,121,2_covid_vaccine_vaccines_deaths,"[covid, vaccine, vaccines, deaths, vaccinated,...",[You’re the one with nothing. HIV- A Virus L...
4,3,113,3_race_black_phenotype_biological,"[race, black, phenotype, biological, racist, s...",[If it matters that much think about it this w...
...,...,...,...,...,...
145,144,10,144_care_addict_romcom_sadly,"[care, addict, romcom, sadly, square, included...","[I care even more., Fine. Fuck it all then. Yo..."
146,145,10,145_aight_arehttpswwwyoutubecomwatchvzgnovemvh...,"[aight, arehttpswwwyoutubecomwatchvzgnovemvhow...",[And [here you are](https://www.youtube.com/wa...
147,146,10,146_cringe_embrassing_clown_fish,"[cringe, embrassing, clown, fish, transition, ...","[None. It just makes u cringe., Given the biol..."
148,147,10,147_lie_lying_framing_fun,"[lie, lying, framing, fun, grade, libleft, dis...",[ Oof... nice straw man there... I know now th...


In [36]:
# get the top words for each topic id (form the topic names)
topic_model_fitted.get_topics()

{-1: [('the', np.float64(0.009648980585927069)),
  ('to', np.float64(0.00900964745006368)),
  ('of', np.float64(0.008728229294142364)),
  ('and', np.float64(0.008676802727406072)),
  ('that', np.float64(0.00853775304029868)),
  ('it', np.float64(0.008349126375051918)),
  ('you', np.float64(0.008204370126110132)),
  ('is', np.float64(0.008018255439060824)),
  ('in', np.float64(0.007840410677819648)),
  ('they', np.float64(0.00757581429546353))],
 0: [('liberal', np.float64(0.01785832046023066)),
  ('conservative', np.float64(0.016136816044106893)),
  ('left', np.float64(0.014996161356335675)),
  ('leftist', np.float64(0.012936205133045244)),
  ('right', np.float64(0.012812919957456672)),
  ('conservatives', np.float64(0.011957939944794539)),
  ('are', np.float64(0.011799667158295474)),
  ('you', np.float64(0.011183770055278461)),
  ('the', np.float64(0.010689509419882181)),
  ('political', np.float64(0.010466061950227076))],
 1: [('pregnancy', np.float64(0.01831910302583223)),
  ('abort

: 

: 